In [17]:
import seaborn as sns
import metapack as mp
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
from tqdm.notebook import tqdm 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()

from demosearch import *
from pathlib import Path



In [13]:
pkg = mp.jupyter.open_source_package()
pkg.set_sys_path()
import  pylib 

def get_cache(pkg):
    return FileCache(Path(pkg.path).parent.joinpath('cache'))

pkg_root = Path(pkg.path).parent

cache = get_cache(pkg)
pkg
 

`civicknowledge.com-percentile-demosearch-1.1.1` Last Update: 2021-05-21T22:46:04

__




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

In [16]:
cache.joinpath()

PosixPath('/Volumes/External/data/radius-collection/staging/civicknowledge.com-percentile-demosearch/data/cache')

In [4]:
bpkg = mp.open_package('http://radius.civicknowledge.com.s3.amazonaws.com/businesslistdatabase.com-us_business-1.1.1.csv')
bpkg


# US Businesses
`businesslistdatabase.com-us_business-1.1.1` Last Update: 2021-04-09T23:19:03

_US Business list, reduced to point position and category_



This package processes a list of US businesses obtained from a
[private source](https://www.businesslistdatabase.com/). The
source data is low quality, so there is a lot of processing
to be done. Most importantly, many records have a conflict between 
the SIC and NAICS codes; these records are removed. 

## Building

The package can take muh longer to build than the timeout for 
running a Jupyter notebook, so the Jupyter notebook caches
intermediate stages of the process. Run the upyter notebook directly
first, then build the package.
## Documentation Links

* [Documentation Page](http://s3.amazonaws.com/radius.civicknowledge.com/businesslistdatabase.com-us_business-1.1.1/index.html) 

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [businesses_cat](http://s3.amazonaws.com/radius.civicknowledge.com/businesslistdatabase.com-us_business-1.1.1/data/businesses_cat.csv)**. Businesses categorized to shop, auto, food, ent

## References
<ul><li> <strong>us_business</string>, <em>data/us_businesses.zip</em>. List of US Businesses</li><li> <strong>block_templ</string>, <em>censusgeo://2020/5/{st}/block</em>. Block url template</li><li> <strong>cbsa</string>, <em>censusgeo://2020/5/US/cbsa</em>. Metro areas</li><li> <strong>states</string>, <em>censusgeo://2020/5/US/state</em>. US States</li><li> <strong>us_geohashes</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-geohash-us.csv#us_geohashes</em>. All 4 digit geohases in the continential US</li><li> <strong>utm_grid</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-mgrs.csv#utm_grid</em>. </li><li> <strong><a href="https://www.census.gov/eos/www/naics/2012NAICS/2012_NAICS_Index_File.xls">naics_index_2012</a></strong>. NAICS index file, 2012</li><li> <strong><a href="https://www.census.gov/eos/www/naics/2012NAICS/2-digit_2012_Codes.xls">naics_index_2012_26</a></strong>. NAICS index file, 2012, 2 to 6 digit codes</li><li> <strong><a href="https://www.census.gov/naics/concordances/1997_NAICS_to_1987_SIC.xls">naics_sic</a></strong>. NAICS to SIC crosswalk, maybe?</li><ul>

In [15]:
%%time

k_src = 'business_scores/combined_source'
if cache.exists(k_src):
    df = cache.get_df(k_src)
else:
    cbsa = bpkg.reference('cbsa').geoframe()
    bc= bpkg.resource('businesses_cat').geoframe()
    t = gpd.sjoin(bc, cbsa.to_crs(4326))
    df = t[~t.geoid.isnull()]
    cache.put_df(k_src, s)
    
dfs = df.sample(500_000)
   
print(len(dfs))
dfs.head() 

Exception ignored in: <function PreparedGeometry.__del__ at 0x7ff190a18e50>
Traceback (most recent call last):
  File "/Users/eric/opt/anaconda3/envs/radius38/lib/python3.8/site-packages/shapely/prepared.py", line 36, in __del__
    lgeos.GEOSPreparedGeom_destroy(self.__geom__)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [3]:
drop_cols = ['agg_hh_income','female','male',
             'over25_college', # corr to agg_income
             'over25_high_school', # corr to total_pop, seniors
             'over25_high_school', # corr to total_pop, seniors
             'housing_rented_college', # Agg_income
             'households', # corr to  total_population
             'households_unmaried', # agg_income, housing_rented_college
             'geometry'
            ]

demo_cols = ['total_population', 'male', 'female', 'over25_college',
       'over25_high_school', 'seniors', 'households', 'households_unmaried',
       'housing_owned_college', 'housing_rented_college', 'agg_income',
       'agg_hh_income']


osm_cols = ['primary', 'secondary', 'tertiary', 'trunk', 'highway',
       'entertain', 'restaurant', 'casual', 'shop', 'bar', 'cafe', 'active',
       'travel', 'food']

cat_cols = set(df.columns)-set(demo_cols)-set(osm_cols)

# High-performing demographics columns
hp_cols = ['seniors',
 'households',
 'housing_owned_college',
 'households_unmaried',
 'housing_rented_college',
 'over25_college',
 'agg_hh_income',
 'male']




NameError: name 'df' is not defined

In [6]:
%%time

k_src = 'business_scores/source/{idx}'
k_score = 'business_scores/scores/{idx}'

tasks = []
chunk_size = 100 #len(s)
rng = list(range(0, len(dfs), chunk_size))
for i in tqdm(rng):
    chunk = dfs.iloc[i:i+chunk_size]
    k = k_src.format(idx=i)
    cache.put_df(k, chunk)
    tasks.append((cache_dir, i, k))
    
cache.get_df('business_scores/source/96600.pkl').head()

NameError: name 'dfs' is not defined

In [7]:

layers = ['total_population','male','female','over25_college','over25_high_school','seniors',
          'households','households_unmaried','housing_owned_college','housing_rented_college',
          'agg_income','agg_hh_income',
          'primary', 'secondary','tertiary', 'trunk', 'highway', 
          'entertain', 'restaurant', 'casual', 'shop','bar','cafe','active','travel', 'food']

def _f(cache_dir, idx, inp):
    from demosearch.exceptions import OutofBoundsError
    from demosearch import FileCache, RasterManager
    
    
    cache = FileCache(cache_dir)
    rm = RasterManager(cache)
    
    k = k_score.format(idx=idx)
   
    if cache.exists(k):
        return k
    
    s = cache.get_df(inp)

    rows = []
    for idx, row in s.iterrows():

        try:
            p = rm.patches( (row.geometry.y, row.geometry.x), layers=layers)
            e = p.score()[0]
           
            e['cbsa'] = row.geoid
            e['naics'] = row.naics
            e['group'] = row.group
            e['geometry'] = row.geometry
           
            rows.append(e)
        
        except OutofBoundsError as err:
            pass
        except Exception as err:
            print(err)
            raise
            pass # Ingore errors
        
    r = gpd.GeoDataFrame(rows)
    
    cache.put_df(k, r)
    return k
 
# Check if the function will pickle
import cloudpickle
pl = cloudpickle.dumps([_f, tasks])
    
#cache.delete('business_scores/scores/1100')
#%time k = _f(*tasks[11])
#s = cache.get_df(k)
#s

In [8]:
def run_mp(f, tasks, desc='', n_cpu=None):
    """Run a function in multiple processes and return the results as a list
    displays a progress bar"""
    from joblib import Parallel, delayed
    from multiprocessing import cpu_count

    if n_cpu is None:
        n_cpu = cpu_count() - 2

    return Parallel(n_jobs=n_cpu)(delayed(f)(*t) for t in tqdm(tasks, desc=desc))

r = run_mp(_f, tasks)


NameError: name 'tqdm' is not defined